0. how can I tell how many tokent a string has before I embed it?: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

In [13]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

1. Obtain the Embeddings: https://github.com/openai/openai-cookbook/blob/main/examples/Get_embeddings_from_dataset.ipynb

In [1]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="I live in chapel hill, NC",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.01816071756184101, -0.020580638200044632, 0.01670197956264019, 0.041930221021175385, -0.032386235892772675, -0.0011145489988848567, 0.01138719916343689, 0.01110449805855751, -0.043965671211481094, -0.05242408812046051, -0.014700455591082573, 0.013411338441073895, -0.04082203283905983, 0.06341550499200821, -0.028993822634220123, 0.025149086490273476, 0.014745688065886497, 0.018081560730934143, -0.016057420521974564, 0.056811608374118805, 0.02797609753906727, -0.007395459804683924, 0.03666067495942116, -0.0577162504196167, -0.010115044191479683, -0.02356596104800701, -0.007124066818505526, -0.01765185408294201, -0.047358084470033646, 0.021202581003308296, -0.025624025613069534, -0.029219983145594597, -0.007435038220137358, -0.05482139065861702, -0.016645438969135284, 0.012235302478075027, 0.0171769168227911, -0.001168968970887363, -0.014983156695961952, -0.010211162269115448, 0.07169298827648163, 0.04591065272688866, -0.0004396001750137657, 0.0003074374108109623, -0.06857196986675262,

Here we show some representative use cases. We will use the Amazon fine-food reviews dataset for the following examples.

In [9]:
from openai import OpenAI
import pandas as pd
import numpy as np

client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [4]:
# load & inspect dataset
input_datapath = "../../examples/data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [6]:
df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)

In [10]:

df = pd.read_csv('output/embedded_1k_reviews.csv')
df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

In [11]:
df.head(10)

,Time,ProductId,UserId,Score,Summary,Text,combined,ada_embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[0.03598902374505997, -0.02119247429072857, -0..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[0.011470186524093151, 0.03424979746341705, -0..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....","[0.0033088172785937786, 0.012649757787585258, ..."
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,"[-0.0028984027449041605, 0.014639723114669323,..."
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,"[0.012089570052921772, -0.056063462048769, 0.0..."
5,1351123200,B008PSM0BQ,A3OUFIMGL2K6RS,4,Good Sauce,This is a good all purpose sauce. Has good fl...,Title: Good Sauce; Content: This is a good all...,"[0.03575918823480606, -0.005432057660073042, -..."
6,1351123200,B008YA1LQK,A9YEAAQVHFUTX,5,Blackcat,Great coffee! Love all Green Mountain coffee ...,Title: Blackcat; Content: Great coffee! Love ...,"[0.002845916198566556, -0.06503486633300781, 0..."
7,1351123200,B001KP6B98,ABWCUS3HBDZRS,5,Excellent product,After scouring every store in town for orange ...,Title: Excellent product; Content: After scour...,"[0.03008199855685234, 0.015235076658427715, -0..."
8,1351123200,B008YA1LQK,A2RSB6FVQ9K9OD,5,Bulk k-Cups,This is the best way to buy coffee for my offi...,Title: Bulk k-Cups; Content: This is the best ...,"[-0.030791310593485832, -0.02117055095732212, ..."
9,1351123200,B001E5E2QI,A23WYVBCNE75X1,3,It's Okay,"Next time, I will buy Gevalia Irish Cream deca...","Title: It's Okay; Content: Next time, I will b...","[-0.041736457496881485, 0.0027115189004689455,..."


2. Reducing embedding dimensions

In [12]:
from openai import OpenAI
import numpy as np

client = OpenAI()

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)


response = client.embeddings.create(
    model="text-embedding-3-small", input="Testing 123", encoding_format="float"
)

cut_dim = response.data[0].embedding[:256]
norm_dim = normalize_l2(cut_dim)

print(norm_dim)

[-6.12177974e-02 -2.97970365e-02  9.81833928e-02 -6.68490052e-02
 -2.29955397e-03  1.40348362e-02  3.44609215e-02  1.36029949e-02
  2.08147424e-02 -1.28256816e-03  1.31763365e-01 -1.01569029e-02
 -5.30646384e-02  5.10263464e-02  6.74708566e-02  7.47603368e-02
 -1.21951927e-02  4.99208359e-02 -7.77313992e-02  6.83345375e-02
  8.47790520e-02  5.24427871e-02  2.97279426e-02 -3.89693415e-02
  4.12149150e-02 -9.32776825e-02 -3.41845417e-02  1.72563701e-02
  5.05081348e-02 -1.05300137e-01  1.04125526e-01 -1.03020018e-01
 -2.38721781e-02  3.95220968e-02  4.94026221e-03 -9.79739513e-04
  2.17971804e-03  5.90067719e-02 -6.59162282e-02 -1.31884282e-02
 -4.96790019e-02 -8.89938171e-02 -8.39930981e-03 -3.45127403e-02
  3.05398048e-02  6.42925038e-02 -9.11357509e-02 -4.22167859e-02
 -1.61076733e-02  1.86382627e-02 -4.13185591e-02  5.60529737e-03
 -4.87807707e-02  1.55462801e-01 -3.16798634e-02  4.59478939e-02
 -8.54699950e-02  6.12177974e-02  1.02190881e-01  3.42018168e-02
  7.08651284e-03 -9.50914

3. Question answering using embeddings-based search  vs directly in Prompt: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

In [ ]:
query = f"""Use the below article on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found, write "I don't know."

Article:
\"\"\"
{wikipedia_article_on_curling}
\"\"\"

Question: Which athletes won the gold medal in curling at the 2022 Winter Olympics?"""

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about the 2022 Winter Olympics.'},
        {'role': 'user', 'content': query},
    ],
    model=GPT_MODEL,
    temperature=0,
)

print(response.choices[0].message.content)

4. Text search using embeddings: https://github.com/openai/openai-cookbook/blob/main/examples/Semantic_text_search_using_embeddings.ipynb

5. Code search using embeddings: https://github.com/openai/openai-cookbook/blob/main/examples/Code_search_using_embeddings.ipynb

6. Recommendations using embeddings: https://github.com/openai/openai-cookbook/blob/main/examples/Recommendation_using_embeddings.ipynb

7. Data visualization in 2D/3D: https://github.com/openai/openai-cookbook/blob/main/examples/Visualizing_embeddings_in_2D.ipynb

8. Regression/predicting number using embeddings: Embedding as a text feature encoder for ML algorithms: https://github.com/openai/openai-cookbook/blob/main/examples/Regression_using_embeddings.ipynb

9. Classification using the embedding features: https://github.com/openai/openai-cookbook/blob/main/examples/Classification_using_embeddings.ipynb

10. Zero-shot classification: https://github.com/openai/openai-cookbook/blob/main/examples/Zero-shot_classification_with_embeddings.ipynb

11. Obtaining user and product embeddings for cold-start recommendation: https://github.com/openai/openai-cookbook/blob/main/examples/User_and_product_embeddings.ipynb

12. Clustering: https://github.com/openai/openai-cookbook/blob/main/examples/Clustering.ipynb